<div style=background-color:#01192d>
&nbsp_<img src=./logo-yellow.svg width=200px>
</div>

You created a new project and want your environment to work directly with the repositories you attached to renku ? Then you're in the right place.

1. Select the repository you want to use as a base for your environment
2. Select the base environment you want to leverage
3. Commit and push to your repo and let the CI build your image
4. Use this image under `Sessions` in your project

> <svg class="octicon octicon-alert mr-2" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="M6.457 1.047c.659-1.234 2.427-1.234 3.086 0l6.082 11.378A1.75 1.75 0 0 1 14.082 15H1.918a1.75 1.75 0 0 1-1.543-2.575Zm1.763.707a.25.25 0 0 0-.44 0L1.698 13.132a.25.25 0 0 0 .22.368h12.164a.25.25 0 0 0 .22-.368Zm.53 3.996v2.5a.75.75 0 0 1-1.5 0v-2.5a.75.75 0 0 1 1.5 0ZM9 11a1 1 0 1 1-2 0 1 1 0 0 1 2 0Z"></path></svg> Warning
>
> It currenly only works on GitHub repositories.

In [ ]:
import json
from subprocess import run
from pathlib import Path
from IPython.display import display, Markdown, clear_output
from ipywidgets import (
    Box,
    Text,
    Dropdown,
    Combobox,
    Checkbox,
    Label,
    Button,
    Layout,
    ButtonStyle,
    Output,
)
from ipyfilechooser import FileChooser
from os import environ

In [ ]:
base_path = Path(environ.get("JUPYTER_SERVER_ROOT", "/home/jovyan/work"))
TEMPLATE = "ghcr.io/salimkayal/renku-devcontainer-template/renku-env"
result = run(
    ["devcontainer", "templates", "metadata", TEMPLATE], text=True, capture_output=True
)
output = json.loads(result.stdout)
options = output["options"]

In [ ]:
# Styles and layouts
form_item_layout = Layout(
    display="flex",
    flex_flow="row",
    justify_content="space-between",
)

form_layout = dict(
    flex_flow="column",
    border="solid 1px",
    align_items="stretch",
    width="700px",
    padding="1%",
)

text_layout = Layout(width="200px")
choice_layout = Layout(width="500px")

apply_form_layout = Layout(**form_layout, display="flex")
commit_form_layout = Layout(**form_layout, display="none")


button_style = ButtonStyle(button_color="#006e58", text_color="#FFF")
button_layout = Layout(width="676px")

In [ ]:
out = Output()

fc = FileChooser(
    base_path,
    show_only_dirs=True,
    sandbox_path=base_path,
)

apply_form_items = [
    Box(
        [
            Label(value="apply template on repo", layout=text_layout),
            Box((fc,), layout=choice_layout),
        ],
        layout=form_item_layout,
    )
]

choices = dict()
for key, values in options.items():
    if values["type"] == "boolean":
        choices[key] = Checkbox(
            value=values.get("default") == "true", layout=choice_layout
        )
        choices[key].style = {"description_width": "0px"}
    elif "enum" in values:
        choices[key] = Dropdown(
            options=values["enum"], value=values.get("default"), layout=choice_layout
        )
    else:
        choices[key] = Combobox(
            options=values["proposals"],
            value=values.get("default"),
            layout=choice_layout,
        )
    apply_form_items.append(
        Box(
            [Label(value=values["description"], Layout=text_layout), choices[key]],
            layout=form_item_layout,
        )
    )

apply_button = Button(
    description="Apply template on repository", layout=button_layout, style=button_style
)


def on_apply_button_clicked(b):
    with out:
        clear_output()
        if (
            fc.value is not None
            and (Path(fc.value) / ".git").is_dir()
            and run(["git", "status"], cwd=fc.value, capture_output=True).returncode
            == 0
        ):
            options_values = {key: choice.value for key, choice in choices.items()}
            apply_button.disabled = True
            template_apply = [
                "devcontainer",
                "templates",
                "apply",
                "-t",
                TEMPLATE,
                "-w",
                fc.value,
                "-a",
                json.dumps(options_values),
            ]
            output = run(template_apply, capture_output=True, text=True)
            if output.returncode == 0:
                clear_output()
                apply_form_layout.display = "none"
                commit_form_layout.display = "flex"
            else:
                print(output.stderr.decode("ASCII"))
                apply_button.disabled = False
        else:
            if fc.value is None:
                print("please enter a git directory")
            elif not (Path(fc.value) / ".git").is_dir():
                print(f"{fc.value} is not a git repository")
            elif gitout := run(["git", "status"], cwd=fc.value, capture_output=True):
                print(gitout.stderr.decode("ASCII"))


apply_button.on_click(on_apply_button_clicked)

apply_form_items.append(apply_button)

apply_form = Box(apply_form_items, layout=apply_form_layout)

In [ ]:
commit_msg = Text(value="feat: added devcontainer", layout=Layout(width="500px"))
commit_button = Button(
    description="rebase, commit and push", layout=button_layout, style=button_style
)


def on_commit_button_clicked(b):
    with out:
        clear_output()
        for command, ignore_errors in (
            (("git", "stash"), False),
            (("git", "pull", "--rebase"), False),
            (("git", "stash", "pop"), True),
            (("git", "add", "."), False),
            (("git", "commit", "-m", commit_msg.value), False),
            (("git", "push"), False),
        ):
            res = run(command, cwd=fc.value, capture_output=True)
            if not ignore_errors and res.returncode != 0:
                print(res.stderr.decode("ASCII"))
                return
        else:
            commit_form_layout.display = "none"
            res = run(
                ("git", "remote", "get-url", "origin"),
                cwd=fc.value,
                capture_output=True,
            )
            if b"github" in res.stdout:
                prefix = "ghcr.io"
                registry_suffix = "/".join(
                    res.stdout.decode("ASCII").strip().split("/")[-2:]
                )
                if registry_suffix[-4:] == ".git":
                    registry_suffix = registry_suffix[:-4]
                display(
                    Markdown(
                        data=f"Use `{prefix}/{registry_suffix}:latest` as your session image in your renku project"
                    )
                )
            else:
                display(
                    Markdown(
                        data="Repository not supported for automated ci image building. **Please provide your own scripts.**"
                    )
                )


commit_button.on_click(on_commit_button_clicked)

commit_form_items = [
    Box(
        [
            Label(value="enter your commit message", layout=Layout(width="200px")),
            commit_msg,
        ],
        layout=form_item_layout,
    ),
    commit_button,
]

commit_form = Box(commit_form_items, layout=commit_form_layout)

In [ ]:
display(apply_form)
display(commit_form)
display(out)